### Imports goes here

In [ ]:
!conda install pandas-profiling --yes

In [2]:
bucket = 'sagemaker-demo1701'
prefix = 'ieee-fraud-detection'
import boto3 #python sdk
import re
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import os
import sagemaker
import warnings
warnings.filterwarnings("ignore")
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer
from sklearn.metrics import roc_auc_score,accuracy_score,f1_score,roc_curve,precision_recall_curve,auc,average_precision_score
role = get_execution_role()
plt.style.use('ggplot')
color_pal = [x['color'] for x in plt.rcParams['axes.prop_cycle']]

### Load Data from S3 bucket

In [4]:
#load train and given data stored in s3
data_location = 's3://{}/{}'.format(bucket, prefix)
train_transaction = pd.read_csv(data_location + '/data/train_transaction.csv')
train_identity = pd.read_csv(data_location + '/data/train_identity.csv')
test_transaction = pd.read_csv(data_location + '/data/test_transaction.csv')
test_identity = pd.read_csv(data_location + '/data/test_identity.csv')

print('train_transaction shape is {}'.format(train_transaction.shape))
print('test_transaction shape is {}'.format(test_transaction.shape))
print('train_identity shape is {}'.format(train_identity.shape))
print('test_identity shape is {}'.format(test_identity.shape))

train_transaction shape is (590540, 394)
test_transaction shape is (506691, 393)
train_identity shape is (144233, 41)
test_identity shape is (141907, 41)


In [7]:
# Here we confirm that all of the transactions in `train_identity`
print(np.sum(train_transaction['TransactionID'].isin(train_identity['TransactionID'].unique())))
print(np.sum(test_transaction['TransactionID'].isin(test_identity['TransactionID'].unique())))
# let's combine the data and work with the whole dataset
train = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
test = pd.merge(test_transaction, test_identity, on='TransactionID', how='left')
print(f'Train dataset has {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'Test dataset has {test.shape[0]} rows and {test.shape[1]} columns.')

144233
141907
Train dataset has 590540 rows and 434 columns.
Test dataset has 506691 rows and 433 columns.


In [8]:
#delete dataframes 
del train_identity, train_transaction, test_identity, test_transaction

### EDA on given dataset

In [10]:
#Create pandas profile for train_transaction dataframe and save as html
profile = pandas_profiling.ProfileReport(train[0:10000],check_recoded = False)

findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.


In [13]:
rejected_variables = profile.get_rejected_variables(threshold=0.9)
profile.to_file(outputfile="output.html")
print ('Total {} variables has to be discarded as perthe profiling results'.format(len(rejected_variables))

SyntaxError: unexpected EOF while parsing (<ipython-input-13-c34225ae487d>, line 3)